## Retrain pre-trained MNIST classifier

Using the model from https://github.com/EN10/KerasMNIST

In [2]:
import numpy as np
from keras.models import load_model
from keras.layers import Dense

Using Theano backend.


In [52]:
from cv_globals import CVROOT

model = load_model(CVROOT + "/weights/cnn.h5")


/anaconda3/envs/chessvision/lib/python3.5/site-packages/keras/models.py:291: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [53]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [54]:
for layer in model.layers[:-1]:
    layer.trainable = False

In [55]:
model.layers.pop()

In [56]:
model.add(Dense(13, activation="softmax", name="final_dense"))

In [61]:
#new_model Sequential()

#= Conv2D(16, (3, 3), input_shape=(64, 64, 1))

model.layers[0]

#model.summary()

In [51]:
# Data stuff

import quilt
import cv2
from sklearn.utils import class_weight
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

labels = {"b": 6, "k": 7, "n": 8, "p": 9, "q": 10, "r": 11, "B": 0,
          "f": 12, "K": 1, "N": 2, "P": 3, "Q": 4, "R": 5}


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    zoom_range=0.05,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=5
)
valid_datagen = ImageDataGenerator(
    rescale=1./255,
)

def install_data():
    quilt.install("gudbrandtandberg/chesspieces", force=True)

def get_data(node, N):
    X = np.zeros((N, 64, 64, 1))
    y = np.zeros((N))
    i = 0
    dirs = node._group_keys()
    for dir_name, dir_node in zip(dirs, node):
        label = labels[dir_name]
        for img in dir_node:
            # load greyscale image
            orig = cv2.imread(img(), 0)
            X[i, :, :, 0] = orig
            y[i] = label
            i += 1
    return X, y

def keras_generator(*, transform=False):
    def _keras_generator(node, paths):

        datagen = train_datagen if transform else valid_datagen

        X, y = get_data(node, len(paths))

        datagen.fit(X)
        y = to_categorical(y)
        return datagen.flow(X, y)
    return _keras_generator

def get_training_generator():
    from quilt.data.gudbrandtandberg import chesspieces as pieces
    return pieces["training"](asa=keras_generator(transform=True))


def get_validation_generator():
    from quilt.data.gudbrandtandberg import chesspieces as pieces
    return pieces["validation"](asa=keras_generator(transform=False))

def labels_only(node, paths):
    _, y = get_data(node, len(paths))
    return y

def get_class_weights():
    from quilt.data.gudbrandtandberg import chesspieces as pieces
    print("Computing class weights")
    y = pieces["training"](asa=labels_only)
    class_weights = class_weight.compute_class_weight('balanced', np.unique(y), y)
    return class_weights


In [49]:
#install_data()
class_weights = get_class_weights()
train_generator = get_training_generator()
valid_generator = get_validation_generator()

Computing class weights


In [50]:
import keras
import time
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint



model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath=CVROOT + "/weights/transfer_model.h5",
                             save_best_only=True,
                             save_weights_only=False)]
                       
start = time.time()
model.fit_generator(generator=train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=100,
                    class_weight=class_weights,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=valid_generator,
                    validation_steps=len(valid_generator))
                       
duration = time.time() - start
print("Training the square classifier took {} minutes and {} seconds".format(
    int(np.floor(duration / 60)), int(np.round(duration % 60))))


Epoch 1/100
255/255 [==============================] - 9s 34ms/step - loss: 2.4268 - acc: 0.2492 - val_loss: 2.2840 - val_acc: 0.3070
Epoch 2/100
255/255 [==============================] - 9s 35ms/step - loss: 2.2957 - acc: 0.2660 - val_loss: 2.1688 - val_acc: 0.3357
Epoch 3/100
255/255 [==============================] - 9s 35ms/step - loss: 2.2262 - acc: 0.2885 - val_loss: 2.0810 - val_acc: 0.3583
Epoch 4/100
255/255 [==============================] - 9s 36ms/step - loss: 2.1661 - acc: 0.3108 - val_loss: 2.0063 - val_acc: 0.3850
Epoch 5/100
255/255 [==============================] - 9s 34ms/step - loss: 2.1158 - acc: 0.3300 - val_loss: 1.9463 - val_acc: 0.4056
Epoch 6/100
255/255 [==============================] - 9s 34ms/step - loss: 2.0805 - acc: 0.3374 - val_loss: 1.8973 - val_acc: 0.4202
Epoch 7/100
255/255 [==============================] - 9s 34ms/step - loss: 2.0470 - acc: 0.3576 - val_loss: 1.8544 - val_acc: 0.4298
Epoch 8/100
255/255 [==============================] - 9s 34ms

Epoch 62/100
255/255 [==============================] - 9s 35ms/step - loss: 1.7400 - acc: 0.4321 - val_loss: 1.4418 - val_acc: 0.5410
Epoch 63/100
255/255 [==============================] - 9s 35ms/step - loss: 1.7329 - acc: 0.4306 - val_loss: 1.4384 - val_acc: 0.5435
Epoch 64/100
255/255 [==============================] - 9s 35ms/step - loss: 1.7516 - acc: 0.4272 - val_loss: 1.4384 - val_acc: 0.5471
Epoch 65/100
255/255 [==============================] - 9s 35ms/step - loss: 1.7414 - acc: 0.4283 - val_loss: 1.4366 - val_acc: 0.5481
Epoch 66/100
255/255 [==============================] - 9s 34ms/step - loss: 1.7327 - acc: 0.4341 - val_loss: 1.4350 - val_acc: 0.5491
Epoch 67/100
255/255 [==============================] - 9s 34ms/step - loss: 1.7327 - acc: 0.4355 - val_loss: 1.4337 - val_acc: 0.5460
Epoch 68/100
255/255 [==============================] - 9s 34ms/step - loss: 1.7406 - acc: 0.4306 - val_loss: 1.4318 - val_acc: 0.5440
Epoch 69/100
255/255 [==============================] -